<a href="https://colab.research.google.com/github/devdhruvper/LegalBrain-VectorSearch-1/blob/main/langchain_scripts/Human_tools/Human_tool_integration_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

intergation of human tool -> sample    --DT

In [18]:
#installing dependecies
!pip install langchain
!pip install fastapi
!pip3 install pinecone-client
!pip install pydantic
!pip install pinecone-text
!pip install google-search-results
from langchain.retrievers import PineconeHybridSearchRetriever
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import Tool , initialize_agent
from langchain import SerpAPIWrapper
from langchain.agents import AgentType
from fastapi.responses import StreamingResponse
import os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install google-search-results
!pip install openai

###setting up enviornment variable

In [12]:
openai_api_key="insert the key"
os.environ["SERPAPI_API_KEY"] = "insert the key"

In [ ]:
# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm = ChatOpenAI(temperature=0.0,openai_api_key=openai_api_key)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

In [14]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [15]:
from langchain.tools import HumanInputRun
tools.append(HumanInputRun())

In [16]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,max_iterations=15
)

###sample prompt

In [17]:
agent.run("who is james cameroon")



> Entering new  chain...
I'm not sure if the input is correct. Did they mean James Cameron, the filmmaker?
Action: Search
Action Input: "James Cameron"
Observation: James Francis Cameron CC is a Canadian filmmaker. A major figure in the post-New Hollywood era, Cameron is considered one of the industry's most innovative filmmakers, regularly making use of novel technologies.
Thought:Yes, it was James Cameron the question was referring to.
Action: human
Action Input: "Can you provide more information about James Cameron?"

Can you provide more information about James Cameron?
titanic

Observation: titanic
Thought:I think the question was asking about James Cameron's most famous film.
Action: Search
Action Input: "James Cameron Titanic"
Observation: James Cameron's "Titanic" is an epic, action-packed romance set against the ill-fated maiden voyage of the R.M.S. Titanic; the pride and joy of the White Star Line and, at the time, the largest moving object ever built. She was the most luxu

'James Cameron is a Canadian filmmaker known for his innovative use of technology and his most famous film is "Titanic".'